# NB03: Contamination vs Functional Potential Models

Compute sample-level functional potential scores and test association with contamination.

Inputs:
- `../data/geochemistry_sample_matrix.tsv`
- `../data/community_taxon_counts.tsv`
- `../data/taxon_bridge.tsv`
- `../data/taxon_functional_features.tsv`

Outputs:
- `../data/site_functional_scores.tsv`
- `../data/model_results.tsv`
- `../figures/contamination_vs_functional_score.png`


In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from scipy.stats import linregress
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

DATA_DIR = Path('../data')
FIG_DIR = Path('../figures')
FIG_DIR.mkdir(parents=True, exist_ok=True)

geo = pd.read_csv(DATA_DIR / 'geochemistry_sample_matrix.tsv', sep='	')
community = pd.read_csv(DATA_DIR / 'community_taxon_counts.tsv', sep='	')
bridge = pd.read_csv(DATA_DIR / 'taxon_bridge.tsv', sep='	')
features = pd.read_csv(DATA_DIR / 'taxon_functional_features.tsv', sep='	')
meta = pd.read_csv(DATA_DIR / 'sample_location_metadata.tsv', sep='	')

for c in ['depth_meter', 'latitude_degree', 'longitude_degree']:
    if c in meta.columns:
        meta[c] = pd.to_numeric(meta[c], errors='coerce')

print('geo:', geo.shape)
print('community:', community.shape)
print('bridge:', bridge.shape)
print('features:', features.shape)
print('meta:', meta.shape)



geo: (108, 49)
community: (41711, 5)
bridge: (8242, 5)
features: (3180, 5)
meta: (108, 7)


In [2]:
metal_keywords = ['uranium', 'chromium', 'nickel', 'zinc', 'copper', 'cadmium', 'lead', 'arsenic', 'mercury']
metal_cols = [c for c in geo.columns if any(k in c.lower() for k in metal_keywords)]
if not metal_cols:
    raise RuntimeError('No contamination columns found in geochemistry_sample_matrix.tsv')

geo_model = geo[['sdt_sample_name'] + metal_cols].copy()
for c in metal_cols:
    geo_model[c] = pd.to_numeric(geo_model[c], errors='coerce')

zparts = []
for c in metal_cols:
    s = np.log1p(geo_model[c])
    std = s.std(ddof=0)
    z = (s - s.mean()) / (std if std else 1)
    zparts.append(z.rename(c + '_z'))

zmat = pd.concat(zparts, axis=1)
geo_model['contamination_index'] = zmat.mean(axis=1, skipna=True)
geo_model = geo_model[['sdt_sample_name', 'contamination_index']].dropna()
print('Samples with contamination_index:', len(geo_model))


Samples with contamination_index: 108


In [3]:
comm = community[['sdt_sample_name', 'genus', 'read_count']].copy()
comm['read_count'] = pd.to_numeric(comm['read_count'], errors='coerce').fillna(0)
comm = comm[comm['read_count'] > 0]

genus_counts = comm.groupby(['sdt_sample_name', 'genus'], as_index=False)['read_count'].sum()
totals = genus_counts.groupby('sdt_sample_name', as_index=False)['read_count'].sum().rename(columns={'read_count':'sample_total'})
genus_counts = genus_counts.merge(totals, on='sdt_sample_name', how='left')
genus_counts['rel_abundance'] = genus_counts['read_count'] / genus_counts['sample_total']
print('Sample-genus abundance rows:', len(genus_counts))


Sample-genus abundance rows: 28001


In [4]:
if 'mapping_mode' not in features.columns:
    features['mapping_mode'] = 'relaxed_all_clades'

bridge_ok = bridge[bridge['mapping_tier'] == 'genus_exact'][['genus', 'genus_norm']].drop_duplicates()

site_scores_all = []
model_results_all = []

for mapping_mode in sorted(features['mapping_mode'].dropna().unique()):
    feat_mode = features[features['mapping_mode'] == mapping_mode].copy()
    feat_wide = feat_mode.pivot_table(index='genus_norm', columns='feature_name', values='feature_value', aggfunc='mean').reset_index()

    genus_feat = genus_counts.merge(bridge_ok, on='genus', how='left').merge(feat_wide, on='genus_norm', how='left')

    feature_cols = ['cog_defense_fraction', 'cog_mobilome_fraction', 'cog_metabolism_fraction']
    for c in feature_cols:
        if c not in genus_feat.columns:
            genus_feat[c] = np.nan
        genus_feat[c] = pd.to_numeric(genus_feat[c], errors='coerce')

    genus_feat['has_feature_mapping'] = genus_feat[feature_cols].notna().any(axis=1)

    for c in feature_cols:
        genus_feat[c] = genus_feat[c].fillna(0.0)

    genus_feat['stress_function_score'] = 0.5 * genus_feat['cog_defense_fraction'] + 0.5 * genus_feat['cog_mobilome_fraction']

    for c in ['cog_defense_fraction', 'cog_mobilome_fraction', 'cog_metabolism_fraction', 'stress_function_score']:
        genus_feat[c + '_weighted'] = genus_feat['rel_abundance'] * genus_feat[c]

    site_scores = genus_feat.groupby('sdt_sample_name', as_index=False)[
        ['cog_defense_fraction_weighted', 'cog_mobilome_fraction_weighted', 'cog_metabolism_fraction_weighted', 'stress_function_score_weighted']
    ].sum()

    coverage = genus_feat.groupby('sdt_sample_name', as_index=False)['rel_abundance'].sum().rename(columns={'rel_abundance': 'total_abundance'})
    mapped_cov = genus_feat[genus_feat['has_feature_mapping']].groupby('sdt_sample_name', as_index=False)['rel_abundance'].sum().rename(columns={'rel_abundance': 'mapped_abundance_fraction'})
    coverage = coverage.merge(mapped_cov, on='sdt_sample_name', how='left').fillna({'mapped_abundance_fraction': 0.0})
    coverage['mapped_abundance_fraction'] = coverage['mapped_abundance_fraction'] / coverage['total_abundance'].replace(0, np.nan)
    coverage['mapped_abundance_fraction'] = coverage['mapped_abundance_fraction'].fillna(0.0)
    coverage['unmapped_abundance_fraction'] = 1.0 - coverage['mapped_abundance_fraction']

    site_scores = site_scores.rename(columns={
        'cog_defense_fraction_weighted': 'site_defense_score',
        'cog_mobilome_fraction_weighted': 'site_mobilome_score',
        'cog_metabolism_fraction_weighted': 'site_metabolism_score',
        'stress_function_score_weighted': 'site_stress_score'
    }).merge(coverage[['sdt_sample_name', 'mapped_abundance_fraction', 'unmapped_abundance_fraction']], on='sdt_sample_name', how='left')

    site_scores['mapping_mode'] = mapping_mode

    model_df = site_scores.merge(geo_model, on='sdt_sample_name', how='inner').merge(
        meta[['sdt_sample_name', 'depth_meter', 'latitude_degree', 'longitude_degree']],
        on='sdt_sample_name', how='left'
    ).dropna(subset=['contamination_index'])

    rows = []
    for y in ['site_defense_score', 'site_mobilome_score', 'site_metabolism_score', 'site_stress_score']:
        d = model_df[['contamination_index', y]].dropna()

        row = {
            'mapping_mode': mapping_mode,
            'outcome': y,
            'status': 'ok',
            'n_samples': len(d),
            'spearman_rho': np.nan,
            'spearman_p': np.nan,
            'permutation_p': np.nan,
            'ols_beta_contamination': np.nan,
            'ols_p_contamination': np.nan,
            'adj_ols_beta_contamination': np.nan,
            'adj_ols_p_contamination': np.nan,
            'adj_ols_n': np.nan,
        }

        if len(d) < 10:
            row['status'] = 'insufficient_samples'
            rows.append(row)
            continue

        if d[y].nunique(dropna=True) < 2 or float(np.nanstd(d[y].to_numpy())) == 0.0:
            row['status'] = 'constant_feature'
            rows.append(row)
            continue

        rho, p_spear = spearmanr(d['contamination_index'], d[y])
        if np.isnan(rho):
            row['status'] = 'invalid_spearman'
            rows.append(row)
            continue

        lr = linregress(d['contamination_index'], d[y])
        row['spearman_rho'] = float(rho)
        row['spearman_p'] = float(p_spear)
        row['ols_beta_contamination'] = float(lr.slope)
        row['ols_p_contamination'] = float(lr.pvalue)

        obs = abs(float(rho))
        perms = 500
        gt = 0
        arr_x = d['contamination_index'].to_numpy()
        arr_y = d[y].to_numpy()
        rng = np.random.default_rng(42)
        for _ in range(perms):
            r, _ = spearmanr(arr_x, rng.permutation(arr_y))
            if not np.isnan(r) and abs(float(r)) >= obs:
                gt += 1
        row['permutation_p'] = (gt + 1) / (perms + 1)

        d_adj = model_df[[y, 'contamination_index', 'depth_meter', 'latitude_degree', 'longitude_degree']].dropna()
        if len(d_adj) >= 20 and d_adj[y].nunique(dropna=True) > 1:
            try:
                fit = smf.ols(f"{y} ~ contamination_index + depth_meter + latitude_degree + longitude_degree", data=d_adj).fit()
                row['adj_ols_beta_contamination'] = float(fit.params.get('contamination_index', np.nan))
                row['adj_ols_p_contamination'] = float(fit.pvalues.get('contamination_index', np.nan))
                row['adj_ols_n'] = int(len(d_adj))
            except Exception:
                pass

        rows.append(row)

    site_scores_all.append(site_scores)
    model_results_all.append(pd.DataFrame(rows))

site_scores_all = pd.concat(site_scores_all, ignore_index=True)
model_results = pd.concat(model_results_all, ignore_index=True)
if len(model_results):
    model_results = model_results.sort_values(['mapping_mode', 'status', 'spearman_p'], na_position='last')

print('Site-score rows:', len(site_scores_all))
print('Model rows:', len(model_results))
print('Mapped-abundance summary by mode:')
print(site_scores_all.groupby('mapping_mode')['mapped_abundance_fraction'].describe().to_string())
model_results



Site-score rows: 216
Model rows: 8
Mapped-abundance summary by mode:
                     count      mean       std       min       25%       50%       75%       max
mapping_mode                                                                                    
relaxed_all_clades   108.0  0.342996  0.183007  0.031131  0.206741  0.303975  0.467964  0.853528
strict_single_clade  108.0  0.342996  0.183007  0.031131  0.206741  0.303975  0.467964  0.853528


,mapping_mode,outcome,status,n_samples,spearman_rho,spearman_p,permutation_p,ols_beta_contamination,ols_p_contamination,adj_ols_beta_contamination,adj_ols_p_contamination,adj_ols_n
0,relaxed_all_clades,site_defense_score,ok,108,0.068240,0.482840,0.463074,0.000888,0.127177,0.001009,0.084181,108
2,relaxed_all_clades,site_metabolism_score,ok,108,-0.014662,0.880288,0.868263,0.000466,0.950499,0.002674,0.720587,108
3,relaxed_all_clades,site_stress_score,ok,108,0.013919,0.886313,0.870259,0.000690,0.673136,0.000985,0.550169,108
1,relaxed_all_clades,site_mobilome_score,ok,108,-0.007459,0.938926,0.938124,0.000492,0.856817,0.000961,0.726901,108
4,strict_single_clade,site_defense_score,ok,108,0.068240,0.482840,0.463074,0.000888,0.127177,0.001009,0.084181,108
6,strict_single_clade,site_metabolism_score,ok,108,-0.014662,0.880288,0.868263,0.000466,0.950499,0.002674,0.720587,108
7,strict_single_clade,site_stress_score,ok,108,0.013919,0.886313,0.870259,0.000690,0.673136,0.000985,0.550169,108
5,strict_single_clade,site_mobilome_score,ok,108,-0.007459,0.938926,0.938124,0.000492,0.856817,0.000961,0.726901,108


In [5]:
# Site-score and model generation moved into previous cell (mapping-mode loop).


In [6]:
# Statistical testing moved into previous cell (mapping-mode loop).


In [7]:
site_scores_all.to_csv(DATA_DIR / 'site_functional_scores.tsv', sep='	', index=False)
model_results.to_csv(DATA_DIR / 'model_results.tsv', sep='	', index=False)

# Diagnostic 1: contamination index distribution
plt.figure(figsize=(6, 4))
plt.hist(geo_model['contamination_index'].dropna(), bins=20)
plt.xlabel('Contamination index (z-score composite)')
plt.ylabel('Sample count')
plt.title('Distribution of contamination index')
plt.tight_layout()
plt.savefig(FIG_DIR / 'contamination_index_distribution.png', dpi=160)
plt.close()

# Diagnostic 2: mapping coverage by mode
genera_total = bridge[['genus']].drop_duplicates().shape[0]
genera_mapped = bridge[bridge['mapping_tier'] == 'genus_exact'][['genus']].drop_duplicates().shape[0]
by_mode = (features[['mapping_mode', 'genus_norm']].drop_duplicates().groupby('mapping_mode')['genus_norm'].nunique())

plt.figure(figsize=(6, 4))
labels = ['enigma_total_genera', 'mapped_genera'] + [f"features_{m}" for m in by_mode.index.tolist()]
vals = [genera_total, genera_mapped] + by_mode.tolist()
plt.bar(labels, vals)
plt.xticks(rotation=20, ha='right')
plt.ylabel('Count')
plt.title('Taxonomy mapping / feature coverage')
plt.tight_layout()
plt.savefig(FIG_DIR / 'mapping_coverage_by_mode.png', dpi=160)
plt.close()

# Main association figure by mapping mode
modes = sorted(site_scores_all['mapping_mode'].dropna().unique())
fig, axes = plt.subplots(1, len(modes), figsize=(6 * max(1, len(modes)), 4), sharey=True)
if len(modes) == 1:
    axes = [axes]

for ax, mode in zip(axes, modes):
    md = site_scores_all[site_scores_all['mapping_mode'] == mode].merge(geo_model, on='sdt_sample_name', how='inner').dropna()
    ax.scatter(md['contamination_index'], md['site_stress_score'], s=16, alpha=0.7)
    if len(md) >= 3:
        m, b = np.polyfit(md['contamination_index'], md['site_stress_score'], 1)
        xs = np.linspace(md['contamination_index'].min(), md['contamination_index'].max(), 100)
        ax.plot(xs, m * xs + b)
    ax.set_title(mode)
    ax.set_xlabel('Contamination index')
axes[0].set_ylabel('Site stress functional score')
fig.suptitle('Contamination vs stress score by mapping mode')
fig.tight_layout()
fig.savefig(FIG_DIR / 'contamination_vs_functional_score.png', dpi=160)
plt.close(fig)

print('Saved:')
print(' -', (DATA_DIR / 'site_functional_scores.tsv').resolve())
print(' -', (DATA_DIR / 'model_results.tsv').resolve())
print(' -', (FIG_DIR / 'contamination_vs_functional_score.png').resolve())
print(' -', (FIG_DIR / 'contamination_index_distribution.png').resolve())
print(' -', (FIG_DIR / 'mapping_coverage_by_mode.png').resolve())
print('\nTop model rows:')
print(model_results.head(12).to_string(index=False) if len(model_results) else 'No model rows produced')


Saved:
 - /home/psdehal/pangenome_science/BERIL-research-observatory/projects/enigma_contamination_functional_potential/data/site_functional_scores.tsv
 - /home/psdehal/pangenome_science/BERIL-research-observatory/projects/enigma_contamination_functional_potential/data/model_results.tsv
 - /home/psdehal/pangenome_science/BERIL-research-observatory/projects/enigma_contamination_functional_potential/figures/contamination_vs_functional_score.png
 - /home/psdehal/pangenome_science/BERIL-research-observatory/projects/enigma_contamination_functional_potential/figures/contamination_index_distribution.png
 - /home/psdehal/pangenome_science/BERIL-research-observatory/projects/enigma_contamination_functional_potential/figures/mapping_coverage_by_mode.png

Top model rows:
       mapping_mode               outcome status  n_samples  spearman_rho  spearman_p  permutation_p  ols_beta_contamination  ols_p_contamination  adj_ols_beta_contamination  adj_ols_p_contamination  adj_ols_n
 relaxed_all_clade